PM10 Nasa processing data

In [40]:
import netCDF4
import numpy as np
print(netCDF4.__version__)
import geopandas as gpd
from shapely.geometry import Point
import fiona

#fiona.config.SHAPE_RESTORE_SHX = 'YES'

1.7.2


In [38]:
nc_file = Dataset('pm10-nasa/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4', mode='r')


In [29]:
print(nc_file)
#print(nc_file.variables.keys())

<class 'netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Contact: http://gmao.gsfc.nasa.gov
    History: Original file generated: Tue Feb 11 23:02:57 2020 GMT
    Filename: MERRA2_400.tavgM_2d_aer_Nx.202001.nc4
    Comment: GMAO filename: d5124_m2_jan10.tavg1_2d_aer_Nx.monthly.202001.nc4
    Source: CVS tag: GEOSadas-5_12_4_p23_sp3_M2-OPS experiment_id: d5124_m2_jan10
    Conventions: CF-1
    Institution: NASA Global Modeling and Assimilation Office
    References: http://gmao.gsfc.nasa.gov
    Format: NetCDF-4/HDF-5
    SpatialCoverage: global
    VersionID: 5.12.4
    TemporalRange: 1980-01-01 -> 2016-12-31
    identifier_product_doi_authority: http://dx.doi.org/
    ShortName: M2TMNXAER
    RangeBeginningDate: 2020-01-01
    RangeEndingDate: 2020-01-31
    GranuleID: MERRA2_400.tavgM_2d_aer_Nx.202001.nc4
    ProductionDateTime: Original file generated: Tue Feb 11 23:02:57 2020 GMT
    LongName: MERRA2 tavg1_2d_aer_Nx: 2d,1-Hourly,Time-averaged,Single

In [30]:
print(nc_file.variables.keys())

dict_keys(['lon', 'lat', 'time', 'BCANGSTR', 'BCCMASS', 'BCEXTTAU', 'BCFLUXU', 'BCFLUXV', 'BCSCATAU', 'BCSMASS', 'DMSCMASS', 'DMSSMASS', 'DUANGSTR', 'DUCMASS', 'DUCMASS25', 'DUEXTT25', 'DUEXTTAU', 'DUFLUXU', 'DUFLUXV', 'DUSCAT25', 'DUSCATAU', 'DUSMASS', 'DUSMASS25', 'OCANGSTR', 'OCCMASS', 'OCEXTTAU', 'OCFLUXU', 'OCFLUXV', 'OCSCATAU', 'OCSMASS', 'SO2CMASS', 'SO2SMASS', 'SO4CMASS', 'SO4SMASS', 'SSANGSTR', 'SSCMASS', 'SSCMASS25', 'SSEXTT25', 'SSEXTTAU', 'SSFLUXU', 'SSFLUXV', 'SSSCAT25', 'SSSCATAU', 'SSSMASS', 'SSSMASS25', 'SUANGSTR', 'SUEXTTAU', 'SUFLUXU', 'SUFLUXV', 'SUSCATAU', 'TOTANGSTR', 'TOTEXTTAU', 'TOTSCATAU', 'Var_BCANGSTR', 'Var_BCCMASS', 'Var_BCEXTTAU', 'Var_BCFLUXU', 'Var_BCFLUXV', 'Var_BCSCATAU', 'Var_BCSMASS', 'Var_DMSCMASS', 'Var_DMSSMASS', 'Var_DUANGSTR', 'Var_DUCMASS', 'Var_DUCMASS25', 'Var_DUEXTT25', 'Var_DUEXTTAU', 'Var_DUFLUXU', 'Var_DUFLUXV', 'Var_DUSCAT25', 'Var_DUSCATAU', 'Var_DUSMASS', 'Var_DUSMASS25', 'Var_OCANGSTR', 'Var_OCCMASS', 'Var_OCEXTTAU', 'Var_OCFLUXU', 'V

In [31]:
file_path = 'pm10-nasa/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4'

layer_number = 71  # Python indexing starts from 0, so layer 72 is index 71
air_density_value = 1.225  # kg/m³

try:
    with Dataset(file_path, 'r') as nc_file:
        # Check if the total mass variables exist
        required_totals = ['SO4SMASS', 'BCSMASS', 'OCCMASS', 'DUSMASS', 'SSSMASS']
        if not all(var in nc_file.variables for var in required_totals):
            missing_totals = [var for var in required_totals if var not in nc_file.variables]
            print(f"Error: The following required total mass variables are missing: {missing_totals}")
        else:
            # Extract the necessary 2D variables
            so4 = nc_file.variables['SO4SMASS'][:]
            bc = nc_file.variables['BCSMASS'][:]
            oc = nc_file.variables['OCCMASS'][:]
            du_total = nc_file.variables['DUSMASS'][:]
            ss_total = nc_file.variables['SSSMASS'][:]

            # Assuming BC and OC are split into hydrophobic and hydrophilic
            bc_phobic = bc * 0.5
            bc_philic = bc * 0.5
            oc_phobic = oc * 0.5
            oc_philic = oc * 0.5

            # Distribute total dust mass based on the PM10 equation coefficients
            total_du_coeff = 1 + 1 + 1 + 0.74
            du001_approx = du_total * (1 / total_du_coeff)
            du002_approx = du_total * (1 / total_du_coeff)
            du003_approx = du_total * (1 / total_du_coeff)
            du004_approx = du_total * (0.74 / total_du_coeff)

            # Distribute total sea salt mass evenly
            ss001_approx = ss_total * 0.25
            ss002_approx = ss_total * 0.25
            ss003_approx = ss_total * 0.25
            ss004_approx = ss_total * 0.25

            # Calculate PM10 concentration
            pm10 = (1.375 * so4 + bc_phobic + bc_philic + oc_phobic + oc_philic +
                    du001_approx + du002_approx + du003_approx + 0.74 * du004_approx +
                    ss001_approx + ss002_approx + ss003_approx + ss004_approx) * air_density_value

            print("Approximate PM10 Concentrations (2D field):")
            print(pm10)

except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'")
except Exception as e:
    print(f"An error occurred: {e}")
'''
try:
    with Dataset(file_path, 'r') as nc_file:
        # Check if the total mass variables exist
        required_totals = ['SO4SMASS', 'BCSMASS', 'OCCMASS', 'DUSMASS', 'SSSMASS']
        if not all(var in nc_file.variables for var in required_totals):
            missing_totals = [var for var in required_totals if var not in nc_file.variables]
            print(f"Error: The following required total mass variables are missing: {missing_totals}")
        else:
            # Extract the necessary variables at the specified layer
            so4 = nc_file.variables['SO4SMASS'][layer_number, :, :]
            bc = nc_file.variables['BCSMASS'][layer_number, :, :]
            oc = nc_file.variables['OCCMASS'][layer_number, :, :]
            du_total = nc_file.variables['DUSMASS'][layer_number, :, :]
            ss_total = nc_file.variables['SSSMASS'][layer_number, :, :]

            # Assuming BC and OC are split into hydrophobic and hydrophilic (you might need to adjust these fractions)
            bc_phobic = bc * 0.5
            bc_philic = bc * 0.5
            oc_phobic = oc * 0.5
            oc_philic = oc * 0.5

            # Distribute total dust mass based on the PM10 equation coefficients
            total_du_coeff = 1 + 1 + 1 + 0.74
            du001_approx = du_total * (1 / total_du_coeff)
            du002_approx = du_total * (1 / total_du_coeff)
            du003_approx = du_total * (1 / total_du_coeff)
            du004_approx = du_total * (0.74 / total_du_coeff)

            # Distribute total sea salt mass evenly (as a placeholder, the equation has 4 modes)
            ss001_approx = ss_total * 0.25
            ss002_approx = ss_total * 0.25
            ss003_approx = ss_total * 0.25
            ss004_approx = ss_total * 0.25

            # Calculate PM10 concentration using the provided equation with approximations and the constant air density
            pm10 = (1.375 * so4 + bc_phobic + bc_philic + oc_phobic + oc_philic +
                    du001_approx + du002_approx + du003_approx + 0.74 * du004_approx +
                    ss001_approx + ss002_approx + ss003_approx + ss004_approx) * air_density_value

            print("Approximate PM10 Concentrations at Layer 72 (using total dust and sea salt, constant air density):")
            print(pm10)

except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'")
except Exception as e:
    print(f"An error occurred: {e}")
'''

Approximate PM10 Concentrations (2D field):
[[[3.80578860e-06 3.80578860e-06 3.80578860e-06 ... 3.80578860e-06
   3.80578860e-06 3.80578860e-06]
  [3.79316120e-06 3.79406740e-06 3.79406740e-06 ... 3.79163120e-06
   3.79163120e-06 3.79163120e-06]
  [3.76665002e-06 3.76665002e-06 3.76665002e-06 ... 3.76665002e-06
   3.76665002e-06 3.76665002e-06]
  ...
  [9.38156420e-07 9.38156420e-07 9.38156420e-07 ... 9.38156420e-07
   9.38156420e-07 9.38156420e-07]
  [9.36620252e-07 9.36544631e-07 9.36544631e-07 ... 9.36790692e-07
   9.36790692e-07 9.36790692e-07]
  [9.35406075e-07 9.35406075e-07 9.35406075e-07 ... 9.35406075e-07
   9.35406075e-07 9.35406075e-07]]]


'\ntry:\n    with Dataset(file_path, \'r\') as nc_file:\n        # Check if the total mass variables exist\n        required_totals = [\'SO4SMASS\', \'BCSMASS\', \'OCCMASS\', \'DUSMASS\', \'SSSMASS\']\n        if not all(var in nc_file.variables for var in required_totals):\n            missing_totals = [var for var in required_totals if var not in nc_file.variables]\n            print(f"Error: The following required total mass variables are missing: {missing_totals}")\n        else:\n            # Extract the necessary variables at the specified layer\n            so4 = nc_file.variables[\'SO4SMASS\'][layer_number, :, :]\n            bc = nc_file.variables[\'BCSMASS\'][layer_number, :, :]\n            oc = nc_file.variables[\'OCCMASS\'][layer_number, :, :]\n            du_total = nc_file.variables[\'DUSMASS\'][layer_number, :, :]\n            ss_total = nc_file.variables[\'SSSMASS\'][layer_number, :, :]\n\n            # Assuming BC and OC are split into hydrophobic and hydrophilic (yo

In [32]:
file_path = 'pm10-nasa/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4'  # Make sure this is the correct path

try:
    with Dataset(file_path, 'r') as nc_file:
        print("Variables in the file:")
        for var_name in nc_file.variables:
            print(var_name)
except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'")
except Exception as e:
    print(f"An error occurred: {e}")

Variables in the file:
lon
lat
time
BCANGSTR
BCCMASS
BCEXTTAU
BCFLUXU
BCFLUXV
BCSCATAU
BCSMASS
DMSCMASS
DMSSMASS
DUANGSTR
DUCMASS
DUCMASS25
DUEXTT25
DUEXTTAU
DUFLUXU
DUFLUXV
DUSCAT25
DUSCATAU
DUSMASS
DUSMASS25
OCANGSTR
OCCMASS
OCEXTTAU
OCFLUXU
OCFLUXV
OCSCATAU
OCSMASS
SO2CMASS
SO2SMASS
SO4CMASS
SO4SMASS
SSANGSTR
SSCMASS
SSCMASS25
SSEXTT25
SSEXTTAU
SSFLUXU
SSFLUXV
SSSCAT25
SSSCATAU
SSSMASS
SSSMASS25
SUANGSTR
SUEXTTAU
SUFLUXU
SUFLUXV
SUSCATAU
TOTANGSTR
TOTEXTTAU
TOTSCATAU
Var_BCANGSTR
Var_BCCMASS
Var_BCEXTTAU
Var_BCFLUXU
Var_BCFLUXV
Var_BCSCATAU
Var_BCSMASS
Var_DMSCMASS
Var_DMSSMASS
Var_DUANGSTR
Var_DUCMASS
Var_DUCMASS25
Var_DUEXTT25
Var_DUEXTTAU
Var_DUFLUXU
Var_DUFLUXV
Var_DUSCAT25
Var_DUSCATAU
Var_DUSMASS
Var_DUSMASS25
Var_OCANGSTR
Var_OCCMASS
Var_OCEXTTAU
Var_OCFLUXU
Var_OCFLUXV
Var_OCSCATAU
Var_OCSMASS
Var_SO2CMASS
Var_SO2SMASS
Var_SO4CMASS
Var_SO4SMASS
Var_SSANGSTR
Var_SSCMASS
Var_SSCMASS25
Var_SSEXTT25
Var_SSEXTTAU
Var_SSFLUXU
Var_SSFLUXV
Var_SSSCAT25
Var_SSSCATAU
Var_SSSMASS
Var_S

In [33]:
from netCDF4 import Dataset

file_path = 'pm10-nasa/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4'

try:
    with Dataset(file_path, 'r') as nc_file:
        if 'SO4SMASS' in nc_file.variables:
            print(f"Dimensions of SO4SMASS: {nc_file.variables['SO4SMASS'].dimensions}")
            print(f"Shape of SO4SMASS: {nc_file.variables['SO4SMASS'].shape}")
        else:
            print("Variable 'SO4SMASS' not found.")
except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'")
except Exception as e:
    print(f"An error occurred: {e}")

Dimensions of SO4SMASS: ('time', 'lat', 'lon')
Shape of SO4SMASS: (1, 361, 576)


In [45]:
try:
    with fiona.Env(SHAPE_RESTORE_SHX='YES'):
        # Then proceed with reading your shapefile using geopandas
        shapefile_path = 'illinois_shape/IL_BNDY_County_Py.shp'  # Make sure this is correct
        illinois_counties = gpd.read_file(shapefile_path)
        # ... your code to process the shapefile ...
except fiona.errors.DriverError as e:
    print(f"Error reading shapefile with Fiona: {e}")
except FileNotFoundError:
    print(f"Error: Shapefile not found at {shapefile_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An unexpected error occurred: Unable to open illinois_shape/IL_BNDY_County_Py.shx or illinois_shape/IL_BNDY_County_Py.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.


In [47]:
file_path = 'pm10-nasa/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path = 'illinois_shape/tl_2023_17_cousub.shp'
air_density_value = 1.225  # kg/m³

# Define approximate Illinois latitude and longitude boundaries
min_lat = 36.9
max_lat = 42.5
min_lon = -91.5
max_lon = -87.5

try:
    with netCDF4.Dataset(file_path, 'r') as nc_file:
        latitudes = nc_file.variables['lat'][:]
        longitudes = nc_file.variables['lon'][:]

        # Find indices of grid points within Illinois latitude and longitude bounds
        lat_indices = np.where((latitudes >= min_lat) & (latitudes <= max_lat))[0]
        lon_indices = np.where((longitudes >= min_lon) & (longitudes <= max_lon))[0]

        # Extract Illinois latitude and longitude coordinates
        illinois_lats = latitudes[lat_indices]
        illinois_lons = longitudes[lon_indices]

        # Extract the 2D aerosol mass mixing ratio variables over the Illinois region
        def extract_illinois_data(var_name):
            if var_name in nc_file.variables:
                var_data = nc_file.variables[var_name][0, lat_indices, lon_indices]  # Assuming time is the first dimension
                return var_data
            else:
                print(f"Warning: Variable '{var_name}' not found.")
                return None

        so4_il = extract_illinois_data('SO4SMASS')
        bc_il = extract_illinois_data('BCSMASS')
        oc_il = extract_illinois_data('OCCMASS')
        du_il = extract_illinois_data('DUSMASS')
        ss_il = extract_illinois_data('SSSMASS')

        if so4_il is not None and bc_il is not None and oc_il is not None and du_il is not None and ss_il is not None:
            # Assuming BC and OC are split into hydrophobic and hydrophilic
            bc_phobic_il = bc_il * 0.5
            bc_philic_il = bc_il * 0.5
            oc_phobic_il = oc_il * 0.5
            oc_philic_il = oc_il * 0.5

            # Distribute total dust mass based on the PM10 equation coefficients
            total_du_coeff = 1 + 1 + 1 + 0.74
            du001_approx_il = du_il * (1 / total_du_coeff)
            du002_approx_il = du_il * (1 / total_du_coeff)
            du003_approx_il = du_il * (1 / total_du_coeff)
            du004_approx_il = du_il * (0.74 / total_du_coeff)

            # Distribute total sea salt mass evenly
            ss001_approx_il = ss_il * 0.25
            ss002_approx_il = ss_il * 0.25
            ss003_approx_il = ss_il * 0.25
            ss004_approx_il = ss_il * 0.25

            # Calculate PM10 concentration over Illinois
            pm10_il = (1.375 * so4_il + bc_phobic_il + bc_philic_il + oc_phobic_il + oc_philic_il +
                       du001_approx_il + du002_approx_il + du003_approx_il + 0.74 * du004_approx_il +
                       ss001_approx_il + ss002_approx_il + ss003_approx_il + ss004_approx_il) * air_density_value

            # Load Illinois counties shapefile
            illinois_counties = gpd.read_file(shapefile_path)

            county_avg_pm10 = {}

            # Iterate through each county
            for index, county in illinois_counties.iterrows():
                county_name = county['NAME']  # Or the appropriate column name for county name
                county_polygon = county['geometry']
                county_pm10_values = []

                # Iterate through the Illinois grid points
                for lat_idx, lat in enumerate(illinois_lats):
                    for lon_idx, lon in enumerate(illinois_lons):
                        point = Point(lon, lat)
                        if county_polygon.contains(point):
                            # Find the corresponding index in the PM10_il array
                            orig_lat_index = lat_indices[lat_idx]
                            orig_lon_index = lon_indices[lon_idx]
                            county_pm10_values.append(pm10_il[orig_lat_index, orig_lon_index])

                if county_pm10_values:
                    avg_pm10 = np.mean(county_pm10_values)
                    county_avg_pm10[county_name] = avg_pm10
                    print(f"Average PM10 concentration in {county_name}: {avg_pm10:.4f}")
                else:
                    print(f"No PM10 data found within {county_name}")

except FileNotFoundError:
    print(f"Error: File not found at '{file_path}' or '{shapefile_path}'")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Unable to open illinois_shape/tl_2023_17_cousub.shx or illinois_shape/tl_2023_17_cousub.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.


In [46]:
try:
    with fiona.Env(SHAPE_RESTORE_SHX='YES'):
        # Then proceed with reading your shapefile using geopandas
        shapefile_path = 'illinois_shape/IL_BNDY_County_Py.shp'  # Make sure this is correct
        illinois_counties = gpd.read_file(shapefile_path)
        # ... your code to process the shapefile ...
except fiona.errors.DriverError as e:
    print(f"Error reading shapefile with Fiona: {e}")
except FileNotFoundError:
    print(f"Error: Shapefile not found at {shapefile_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An unexpected error occurred: Unable to open illinois_shape/IL_BNDY_County_Py.shx or illinois_shape/IL_BNDY_County_Py.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.
